In [1]:
# 1. 패키지 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def load_data():
    승하차_파일 = "../../data/결과/승하차/통합/2호선_승하차인원_통합.csv"
    혼잡도_파일 = "../../data/결과/혼잡도/통합/2호선_혼잡도_통합.csv"
    승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
    혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
    # 시간컬럼명 0패딩 통일
    def fix_timecols(cols):
        return [c if ':' not in c else c.zfill(5) for c in cols]
    승하차_df.columns = fix_timecols(승하차_df.columns)
    혼잡도_df.columns = fix_timecols(혼잡도_df.columns)
    return 승하차_df, 혼잡도_df


In [3]:
# 3. 승하차 데이터를 상행/하행으로 2배 복제
def duplicate_for_directions(승하차):
    dfs = []
    for direction in ["상행", "하행"]:
        tmp = 승하차.copy()
        tmp['방향'] = direction
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)


In [4]:
# 전처리 함수
def standardize_columns(df, do_direction=True):
    df["평일주말"] = df["평일주말"].astype(str).str.strip()
    df["구분"] = df["구분"].astype(str).str.strip()
    df["역번호"] = df["역번호"].astype(str).str.strip()
    if do_direction and "방향" in df.columns:
        df["방향"] = df["방향"].astype(str).str.strip()
    return df




In [5]:
# 5. AI 딥러닝 학습 데이터 생성
def build_dl_dataset(승하차, 혼잡도, 정원=2000):
    rows = []
    시간컬럼들 = [col for col in 승하차.columns if ':' in col]
    fail, total = 0, 0
    for idx, row in 승하차.iterrows():
        for col in 시간컬럼들:
            total += 1
            hour = int(col.split(":")[0])
            평일주말 = 1 if row["평일주말"] == "주말" else 0
            상행 = 1 if row["방향"] == "상행" else 0
            승차 = 1 if row["구분"] == "승차" else 0
            matched = 혼잡도[
                (혼잡도["역번호"].astype(str).str.strip() == row["역번호"]) &
                (혼잡도["평일주말"].astype(str).str.strip() == row["평일주말"]) &
                (혼잡도["구분"].astype(str).str.strip() == row["방향"])
            ]
            if matched.empty or col not in 혼잡도.columns:
                fail += 1
                continue
            try:
                congestion = float(matched.iloc[0][col])
                승하차인원 = float(row[col])
            except Exception as e:
                fail += 1
                continue
            y = int(congestion * 정원 / 100)
            rows.append([
                int(row["역번호"]), hour, 평일주말, 상행, 승차, 승하차인원, congestion, y
            ])
    print(f"> 학습데이터 생성: {len(rows)}행, 실패 {fail} / 총시도 {total}")
    cols = ["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도", "target"]
    return pd.DataFrame(rows, columns=cols)

In [6]:
# 6. 전체 파이프라인 (학습+예측)
승하차, 혼잡도 = load_data()
승하차_expanded = duplicate_for_directions(승하차)
승하차_expanded = standardize_columns(승하차_expanded)
혼잡도 = standardize_columns(혼잡도)
df = build_dl_dataset(승하차_expanded, 혼잡도)
print(df.head())
if df.shape[0]==0:
    raise ValueError("학습데이터 생성 실패! 컬럼 또는 값 일치 여부 확인 필요.")


> 학습데이터 생성: 13016520행, 실패 131480 / 총시도 13148000
   역번호  hour  평일주말  상행  승차  승하차인원   혼잡도  target
0  201     5     1   1   1   39.0   5.2     104
1  201     6     1   1   1   51.0   9.2     184
2  201     7     1   1   1   95.0   9.9     198
3  201     8     1   1   1  104.0  18.0     360
4  201     9     1   1   1  129.0  21.0     420


In [7]:
# 7. 입력/정규화/트레인테스트 분리
X = df[["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도"]].values
y = df["target"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [8]:
# 8. 딥러닝 모델 정의&학습
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=40,
    batch_size=64,
    verbose=2
)


Epoch 1/40


c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


146436/146436 - 122s - 832us/step - loss: 137383.0000 - mae: 265.9721 - val_loss: 136610.6875 - val_mae: 265.1388
Epoch 2/40
146436/146436 - 122s - 835us/step - loss: 136507.9375 - mae: 265.1344 - val_loss: 136613.7344 - val_mae: 265.1971
Epoch 3/40
146436/146436 - 123s - 842us/step - loss: 136509.6719 - mae: 265.1288 - val_loss: 136609.1094 - val_mae: 264.7335
Epoch 4/40
146436/146436 - 123s - 841us/step - loss: 136507.6094 - mae: 265.1333 - val_loss: 136608.6250 - val_mae: 264.7450
Epoch 5/40
146436/146436 - 122s - 834us/step - loss: 136506.8906 - mae: 265.1308 - val_loss: 136606.8906 - val_mae: 264.7937
Epoch 6/40
146436/146436 - 123s - 843us/step - loss: 136507.7344 - mae: 265.1339 - val_loss: 136637.7812 - val_mae: 265.4820
Epoch 7/40
146436/146436 - 123s - 837us/step - loss: 136508.6250 - mae: 265.1330 - val_loss: 136636.8594 - val_mae: 264.4587
Epoch 8/40
146436/146436 - 123s - 843us/step - loss: 136509.7656 - mae: 265.1317 - val_loss: 136604.8125 - val_mae: 264.9417
Epoch 9/40


In [9]:
# 학습 직후 (fit 다음 셀)
model.save("my_dl_model.2line.h5")
import joblib
joblib.dump(scaler, "my_scaler.2line.pkl")


['my_scaler.2line.pkl']

In [10]:
# 9. 예측 함수
def dl_predict(역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도):
    arr = np.array([[역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도]])
    arr_scaled = scaler.transform(arr)
    pred = model.predict(arr_scaled)
    return int(pred[0][0])


In [11]:
def user_predict_smart():
    # 입력
    역명 = input("역명: ")
    호선 = input("호선: ")
    방향 = input("방향 (상행/하행): ").strip()
    평일주말_str = input("요일 (평일/주말): ").strip()
    시간 = input("시간 (HH:MM): ").strip()

    # 시간 가공
    hour = int(시간.split(":")[0])
    평일주말 = 1 if 평일주말_str == "주말" else 0
    상행 = 1 if 방향 == "상행" else 0

    # 역번호 자동 조회
    # 최신화 : 승하차/혼잡도 둘 다에 맞추려면 승하차 or 혼잡도 데이터(ex. 승하차_expanded)에서 조회
    # 보통 승하차 데이터에 역명, 호선이 있으므로:
    df_ref = 승하차_expanded
    후보 = df_ref[(df_ref["역명"] == 역명) & (df_ref["호선"].astype(str) == str(호선))]
    if 후보.empty:
        print(f"해당 역명/호선이 데이터에 없습니다.")
        return
    역번호 = int(후보.iloc[0]["역번호"])

    # 승하차 값 탐색
    row_승 = df_ref[
        (df_ref["역번호"] == str(역번호)) &
        (df_ref["방향"] == 방향) &
        (df_ref["평일주말"] == 평일주말_str) &
        (df_ref["구분"] == "승차")
    ]
    row_하 = df_ref[
        (df_ref["역번호"] == str(역번호)) &
        (df_ref["방향"] == 방향) &
        (df_ref["평일주말"] == 평일주말_str) &
        (df_ref["구분"] == "하차")
    ]
    시간컬럼 = 시간 if 시간 in df_ref.columns else f"{hour:02d}:00"
    # 승차/하차 인원 조회(둘 다 있는 경우 합쳐도 되고, 하나만 써도 됨)
    if not row_승.empty and 시간컬럼 in row_승.columns:
        승차인원 = float(row_승.iloc[0][시간컬럼])
    else:
        승차인원 = 0
    if not row_하.empty and 시간컬럼 in row_하.columns:
        하차인원 = float(row_하.iloc[0][시간컬럼])
    else:
        하차인원 = 0
    승하차인원 = 승차인원 + 하차인원 # 혹은 필요에 따라 방법 선택

    # 혼잡도 탐색
    row_혼 = 혼잡도[
        (혼잡도["역번호"] == str(역번호)) &
        (혼잡도["구분"] == 방향) &
        (혼잡도["평일주말"] == 평일주말_str)
    ]
    if not row_혼.empty and 시간컬럼 in row_혼.columns:
        혼잡도_val = float(row_혼.iloc[0][시간컬럼])
    else:
        혼잡도_val = 0

    # 예측
    pred = dl_predict(
        역번호=역번호,
        hour=hour,
        평일주말=평일주말,
        상행=상행,
        승차=1,  # 승차만 예측할지, 승/하차인원합 예측할지는 논리 선택!
        승하차인원=승하차인원,
        혼잡도=혼잡도_val
    )

    # 출력
    print(f"예측: {호선}호선 {역명}({역번호}) {방향} {평일주말_str} {시간}")
    print(f"  승하차 인원(입력값): {승하차인원}, 혼잡도(%): {혼잡도_val:.1f}")
    print(f"  딥러닝 기반 예측 열차 내 인원: {pred} 명")


In [12]:
# 실행 예시
user_predict_smart()

ValueError: invalid literal for int() with base 10: ''